신경망은 데이터에 대한 연산을 수행하는 계층(layer)/모듈(module)로 구성되어 있습니다. torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공합니다. PyTorch의 모든 모듈은 nn.Module 의 하위 클래스(subclass) 입니다. 신경망은 다른 모듈(계층; layer)로 구성된 모듈입니다. 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있습니다.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
print(logits.shape)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

torch.Size([1, 10])
Predicted class: tensor([3], device='cuda:0')


In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten 계층을 초기화하여 각 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환합니다. (dim=0의 미니배치 차원은 유지됩니다.)

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-3.9093e-01,  2.2060e-01,  1.5328e-01, -1.5238e-02,  2.1586e-01,
          2.8610e-02,  2.9991e-01, -1.2572e-01, -5.3425e-02,  1.0204e-01,
         -6.2200e-01, -3.4402e-01, -4.6535e-01, -9.7609e-01, -2.5768e-01,
         -9.2474e-02,  1.9585e-01, -2.2891e-02,  3.2511e-01, -5.4161e-01],
        [-8.4140e-01,  3.1647e-01,  2.9102e-01, -1.8066e-01, -3.1976e-01,
         -1.7454e-02, -8.5660e-02,  2.0851e-01, -2.7337e-01,  2.5412e-01,
         -4.7775e-01, -7.2067e-02, -3.7227e-02, -7.1914e-01, -3.3659e-01,
         -3.9443e-01,  3.6950e-03, -3.8255e-01,  2.9992e-01, -5.5513e-01],
        [-5.4929e-01,  9.2412e-02,  3.3756e-01, -7.1007e-03,  5.8333e-02,
         -1.3316e-01, -9.3283e-02,  1.7217e-01, -3.1682e-01,  2.0244e-01,
         -5.6131e-01, -4.7658e-04, -8.5829e-02, -8.4687e-01, -3.8955e-01,
         -2.5035e-01, -1.2940e-01, -9.5935e-02, -3.8858e-02, -6.2111e-01]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.2206, 0.1533, 0.0000, 0.2159,

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달될 ([-infty, infty] 범위의 원시 값(raw value)인) logits 를 반환합니다. logits는 모델의 각 분류(class)에 대한 예측 확률을 나타내도록 [0, 1] 범위로 비례하여 조정(scale)됩니다. dim 매개변수는 값의 합이 1이 되는 차원을 나타냅니다

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0324, -0.0342, -0.0233,  ..., -0.0270,  0.0148,  0.0009],
        [ 0.0063,  0.0120,  0.0209,  ...,  0.0321, -0.0104, -0.0323]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0169, -0.0118], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0284,  0.0309,  0.0181,  ...,  0.0304, -0.0260,  0.0036],
        [ 0.0010, -0.0288,  0.0166,  ...,  0.0373,  0.0004,  0.0065]],
       device='cuda:0', grad_fn=<Sl